In [321]:
import requests
import json
import requests
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime
from IPython import get_ipython

### Check Data Lastest Date

In [ ]:
# Get the needed API endpoint

summary_url = "https://banks.data.fdic.gov/api/summary?fields=STNAME%2CYEAR%2CINTINC%2CEINTEXP%2CNIM%2CNONII%2CNONIX%2CELNATR%2CITAXR%2CIGLSEC%2CITAX%2CEXTRA%2CNETINC&sort_by=YEAR&sort_order=DESC&limit=10000&offset=0&format=json&download=false"

location_url = "https://banks.data.fdic.gov/api/locations?fields=NAME%2CUNINUM%2CFI_UNINUM%2CCITY%2CSTNAME%2CZIP%2CCOUNTY&sort_by=NAME&sort_order=ASC&limit=10000&offset=0&format=json&download=false&filename=data_file"

institution_url = "https://banks.data.fdic.gov/api/institutions?filters=ACTIVE%3A1&fields=UNINUM%2CZIP%2CCITY%2CCOUNTY%2CSTNAME%2CSTALP%2CNAME%2CACTIVE%2CASSET%2CDEP&sort_by=NAME&sort_order=ASC&limit=10000&offset=0&format=json&download=false&filename=data_file"

In [ ]:
# Get request to the API

response_summary = requests.get(summary_url)
response_location = requests.get(location_url)
response_institution = requests.get(institution_url) # Active flag is 1

In [ ]:
# Check if the requests are successful and parse the dates

for i in [response_summary, response_location, response_institution]:
    if i.status_code == 200:
        data = i.json()
        create_timestamp = data["meta"]["index"]["createTimestamp"]
        print(f"Successfully fetched the data for the date {create_timestamp}")
    else:
        print(f"Failed to fetch data.")

In [335]:
last_processed_date = datetime(2025, 2, 12)

location_date = datetime.strptime(response_location.json()["meta"]["index"]["createTimestamp"], "%Y-%m-%dT%H:%M:%SZ")
institution_date = datetime.strptime(response_institution.json()["meta"]["index"]["createTimestamp"], "%Y-%m-%dT%H:%M:%SZ")

### Extract Banks Data

In [318]:
def fetch_all_data(api_url, params, limit=10000):
    
    all_data = []
    params["limit"] = limit
    params["offset"] = 0
    
    while True:
        # Make the request
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an error if the request fails
        
        data = response.json()
        
        # Check if there are records in the response
        if "data" in data and data["data"]:
            all_data.extend(data["data"])  # Append the new data
            print(f"Retrieved {len(data['data'])} records. Total so far: {len(all_data)}")
            
            # Break the loop if fewer than the limit of records were returned
            if len(data["data"]) < limit:
                break
            
            # Increment the offset for the next batch
            params["offset"] += limit
        else:
            print("No more data to fetch.")
            break

    # Convert the list of JSON data to a pandas DataFrame
    df = pd.json_normalize(all_data, sep="_")
    return df


#### Locations

In [ ]:
# If last processed date falls behind, need to re-fetch the data 

if (location_date > last_processed_date) or (institution_date > last_processed_date):
    print("Running all cells below...")
    shell = get_ipython()
    shell.run_line_magic("run", "-i AllCellsBelow")

In [319]:
# API 
api_url = "https://banks.data.fdic.gov/api/locations"
params = {
    "fields": "NAME,UNINUM,FI_UNINUM,CITY,STNAME,ZIP,COUNTY",
    "sort_by": "NAME",
    "sort_order": "ASC",
    "format": "json"
}

# Fetch all data and convert to DataFrame
df_locations = fetch_all_data(api_url, params)

# Display the first few rows of the DataFrame
print(df_locations.shape)
df_locations.head()

Retrieved 10000 records. Total so far: 10000
Retrieved 10000 records. Total so far: 20000
Retrieved 10000 records. Total so far: 30000
Retrieved 10000 records. Total so far: 40000
Retrieved 10000 records. Total so far: 50000
Retrieved 10000 records. Total so far: 60000
Retrieved 10000 records. Total so far: 70000
Retrieved 8872 records. Total so far: 78872
(78872, 9)


,score,data_ZIP,data_CITY,data_FI_UNINUM,data_STNAME,data_COUNTY,data_NAME,data_UNINUM,data_ID
0,1,62230,Breese,9231,Illinois,Clinton,1NB Bank,223055,223055
1,1,62231,Carlyle,9231,Illinois,Clinton,1NB Bank,232078,232078
2,1,62216,Aviston,9231,Illinois,Clinton,1NB Bank,466427,466427
3,1,62231,Carlyle,9231,Illinois,Clinton,1NB Bank,9231,9231
4,1,63376,Saint Peters,429739,Missouri,St. Charles,1st Advantage Bank,429739,429739


#### Institution

In [362]:
# API parameters for the new API
api_url = "https://banks.data.fdic.gov/api/institutions"
params = {
    "filters": "ACTIVE:1",
    "fields": "UNINUM,REPDTE,ZIP,CITY,COUNTY,STNAME,STALP,NAME,ACTIVE,ASSET,DEP",
    "sort_by": "REPDTE",
    "sort_order": "DESC",
    "format": "json"
}

# Fetch all data and convert to DataFrame
df_institutions = fetch_all_data(api_url, params)

# df_institutions["UNINUM"] = df_institutions["UNINUM"].astype(int)

# Display the first few rows of the DataFrame
print(df_institutions.shape)
df_institutions.head()


Retrieved 4490 records. Total so far: 4490
(4490, 13)


,score,data_ZIP,data_CITY,data_ACTIVE,data_REPDTE,data_STNAME,data_ASSET,data_STALP,data_DEP,data_COUNTY,data_NAME,data_UNINUM,data_ID
0,0,53946,Markesan,1,09/30/2024,Wisconsin,242674.0,WI,202952.0,Green Lake,Ergo Bank,6394,10004
1,0,53566,Monroe,1,09/30/2024,Wisconsin,452264.0,WI,344870.0,Green,Woodford State Bank,6400,10011
2,0,54909,Almond,1,09/30/2024,Wisconsin,210139.0,WI,173856.0,Portage,The Portage County Bank,6401,10012
3,0,54757,New Auburn,1,09/30/2024,Wisconsin,204089.0,WI,172964.0,Chippewa,Security Bank,6404,10015
4,0,54935,Fond Du Lac,1,09/30/2024,Wisconsin,2865827.0,WI,2323421.0,Fond Du Lac,National Exchange Bank and Trust,6419,10044


In [316]:
print(df_summary.shape)
df_summary.head()

(7865, 14)


,ITAX,EINTEXP,EXTRA,ELNATR,STNAME,INTINC,NETINC,NONIX,NIM,YEAR,ITAXR,IGLSEC,NONII,ID
0,32827.0,102349.0,0.0,4206.0,Alaska,412965.0,103186,227489.0,310616.0,2023,136536,-523.0,57615.0,CB_2023_AK
1,727856.0,2498542.0,0.0,644948.0,Alabama,9955600.0,3012921,5563262.0,7457058.0,2023,3745419,-4642.0,2496571.0,CB_2023_AL
2,432747.0,2902779.0,0.0,366774.0,Arkansas,8343451.0,1898949,4321038.0,5440672.0,2023,2346191,-14473.0,1593331.0,CB_2023_AR
3,0.0,0.0,0.0,0.0,American Samoa,0.0,0,0.0,0.0,2023,0,0.0,0.0,CB_2023_AS
4,212045.0,1707821.0,0.0,65783.0,Arizona,4178894.0,710148,1793633.0,2471073.0,2023,964990,-42796.0,353333.0,CB_2023_AZ


In [315]:
print(df_location.shape)
df_location.head()

(10000, 8)


,ZIP,CITY,FI_UNINUM,STNAME,COUNTY,NAME,UNINUM,ID
0,62230,Breese,9231,Illinois,Clinton,1NB Bank,223055,223055
1,62231,Carlyle,9231,Illinois,Clinton,1NB Bank,232078,232078
2,62216,Aviston,9231,Illinois,Clinton,1NB Bank,466427,466427
3,62231,Carlyle,9231,Illinois,Clinton,1NB Bank,9231,9231
4,63376,Saint Peters,429739,Missouri,St. Charles,1st Advantage Bank,429739,429739


In [359]:
df_institution["UNINUM"] = df_institution["UNINUM"].astype(int)
# df_institution["ID"] = df_institution["ID"].astype(int)

print(df_institution.shape)
df_institution.head()

(4490, 11)


,ZIP,CITY,ACTIVE,STNAME,ASSET,STALP,DEP,COUNTY,NAME,UNINUM,ID
0,62231,Carlyle,1,Illinois,301003.0,IL,254473.0,Clinton,1NB Bank,9231,14761
1,63376,Saint Peters,1,Missouri,181448.0,MO,162837.0,St. Charles,1st Advantage Bank,429739,57899
2,74035,Hominy,1,Oklahoma,47473.0,OK,44326.0,Osage,1st Bank in Hominy,2688,4122
3,08243,Sea Isle City,1,New Jersey,301759.0,NJ,219399.0,Cape May,1st Bank of Sea Isle City,43201,30367
4,85364,Yuma,1,Arizona,577947.0,AZ,518166.0,Yuma,1st Bank Yuma,360755,57298


In [311]:
# Merge three dataframes
merged_bank_0 = pd.merge(df_institution, df_location, left_on="UNINUM", right_on="FI_UNINUM", how="inner")

# Add a column to flag "Bank"
merged_bank_0["Type"] = "Bank"
  
print(merged_bank_0.shape)
merged_bank_0.head()

(10000, 20)


,ZIP_x,CITY_x,ACTIVE,STNAME_x,ASSET,STALP,DEP,COUNTY_x,NAME_x,UNINUM_x,ID_x,ZIP_y,CITY_y,FI_UNINUM,STNAME_y,COUNTY_y,NAME_y,UNINUM_y,ID_y,Type
0,62231,Carlyle,1,Illinois,301003.0,IL,254473.0,Clinton,1NB Bank,9231,14761,62230,Breese,9231,Illinois,Clinton,1NB Bank,223055,223055,Bank
1,62231,Carlyle,1,Illinois,301003.0,IL,254473.0,Clinton,1NB Bank,9231,14761,62231,Carlyle,9231,Illinois,Clinton,1NB Bank,232078,232078,Bank
2,62231,Carlyle,1,Illinois,301003.0,IL,254473.0,Clinton,1NB Bank,9231,14761,62216,Aviston,9231,Illinois,Clinton,1NB Bank,466427,466427,Bank
3,62231,Carlyle,1,Illinois,301003.0,IL,254473.0,Clinton,1NB Bank,9231,14761,62231,Carlyle,9231,Illinois,Clinton,1NB Bank,9231,9231,Bank
4,63376,Saint Peters,1,Missouri,181448.0,MO,162837.0,St. Charles,1st Advantage Bank,429739,57899,63376,Saint Peters,429739,Missouri,St. Charles,1st Advantage Bank,429739,429739,Bank


### Extract Credit Union Data

In [55]:
## Mar 2024

# Define file path
file_path = "515_Mar2024.xlsx"

# Read specific sheets and specific columns
sheet_1 = "Total Accounts"  
sheet_2 = "Shares and Deposits"
sheet_3 = "ProfileGenInfo"
columns_to_read_1 = ["Charter", "010"]  
columns_to_read_2 = ["Charter", "018"]
columns_to_read_3 = ["CUNumber", "CUName", "City", "State"]

# Read the data into a DataFrame
df_1 = pd.read_excel(file_path, sheet_name=sheet_1, usecols=columns_to_read_1)
df_2 = pd.read_excel(file_path, sheet_name=sheet_2, usecols=columns_to_read_2)
df_3 = pd.read_excel(file_path, sheet_name=sheet_3, usecols=columns_to_read_3)


# Rename the columns
ndf_1 = df_1.rename(columns={"010": "Total Assets"})
ndf_2 = df_2.rename(columns={"018": "Total Shares and Deposits"})

# Merge three dataframes
merged_df_0 = pd.merge(ndf_1, ndf_2, on="Charter", how="inner")
merged_df = pd.merge(merged_df_0, df_3, left_on="Charter", right_on="CUNumber", how="inner")

# Add a column to flag "CU"
merged_df['Type'] = 'CU'

print(merged_df.head())

   Charter  Total Assets  Total Shares and Deposits  CUNumber  \
0      566    3319681727                 2557819422       566   
1      594     419766565                  380120801       594   
2     1034      64727010                   50306580      1034   
3     1074    1540312535                 1274446370      1074   
4     1204     109996148                  101806576      1204   

                 CUName             City State Type  
0              NUVISION  HUNTINGTON BEAC    CA   CU  
1              PASADENA         Pasadena    CA   CU  
2  OLIVE VIEW EMPLOYEES           SYLMAR    CA   CU  
3     FARMERS INSURANCE          Burbank    CA   CU  
4                RANCHO           DOWNEY    CA   CU  


In [381]:
## June 2024

# Define file path
file_path = "585_Jun2024.xlsx"

# Read specific sheets and specific columns
sheet_1 = "Total Accounts"  
sheet_2 = "Shares and Deposits"
sheet_3 = "ProfileGenInfo"
columns_to_read_1 = ["Charter", "010"]  
columns_to_read_2 = ["Charter", "018"]
columns_to_read_3 = ["CUNumber", "CUName", "City", "State"]

# Read the data into a DataFrame
df_1 = pd.read_excel(file_path, sheet_name=sheet_1, usecols=columns_to_read_1)
df_2 = pd.read_excel(file_path, sheet_name=sheet_2, usecols=columns_to_read_2)
df_3 = pd.read_excel(file_path, sheet_name=sheet_3, usecols=columns_to_read_3)


# Rename the columns
ndf_1 = df_1.rename(columns={"010": "Total Assets"})
ndf_2 = df_2.rename(columns={"018": "Total Shares and Deposits"})

# Merge three dataframes
merged_df_0 = pd.merge(ndf_1, ndf_2, on="Charter", how="inner")
merged_df_june = pd.merge(merged_df_0, df_3, left_on="Charter", right_on="CUNumber", how="inner")

# Add a column to flag "CU"
merged_df_june['Type'] = 'CU'

print(merged_df_june.head())

   Charter  Total Assets  Total Shares and Deposits  CUNumber  \
0        1      11109125                    9758484         1   
1        6     274014841                  239261238         6   
2       12      60169418                   55132588        12   
3       13    1074739920                  926710410        13   
4       16       9617599                    8334581        16   

                      CUName State         City Type  
0  MORRIS SHEPPARD TEXARKANA    TX    TEXARKANA   CU  
1  THE NEW ORLEANS FIREMEN'S    LA     Metairie   CU  
2             FRANKLIN TRUST    CT     Hartford   CU  
3             EFCU FINANCIAL    LA  Baton Rouge   CU  
4                    WOODMEN    NE        OMAHA   CU  


In [382]:
## Sep 2024

# Define file path
file_path = "566_Sep2024.xlsx"

# Read specific sheets and specific columns
sheet_1 = "Total Accounts"  
sheet_2 = "Shares and Deposits"
sheet_3 = "ProfileGenInfo"
columns_to_read_1 = ["Charter", "010"]  
columns_to_read_2 = ["Charter", "018"]
columns_to_read_3 = ["CUNumber", "CUName", "City", "State"]

# Read the data into a DataFrame
df_1 = pd.read_excel(file_path, sheet_name=sheet_1, usecols=columns_to_read_1)
df_2 = pd.read_excel(file_path, sheet_name=sheet_2, usecols=columns_to_read_2)
df_3 = pd.read_excel(file_path, sheet_name=sheet_3, usecols=columns_to_read_3)


# Rename the columns
ndf_1 = df_1.rename(columns={"010": "Total Assets"})
ndf_2 = df_2.rename(columns={"018": "Total Shares and Deposits"})

# Merge three dataframes
merged_df_0 = pd.merge(ndf_1, ndf_2, on="Charter", how="inner")
merged_df_sep = pd.merge(merged_df_0, df_3, left_on="Charter", right_on="CUNumber", how="inner")

# Add a column to flag "CU"
merged_df_sep['Type'] = 'CU'

print(merged_df_sep.head())

   Charter  Total Assets  Total Shares and Deposits  CUNumber  \
0      566    3469113230                 2731790560       566   
1      594     364712289                  324358927       594   
2     1034      63443173                   48931060      1034   
3     1074    1461148571                 1229322187      1074   
4     1204     108016124                   99260328      1204   

                 CUName             City State Type  
0              NUVISION  HUNTINGTON BEAC    CA   CU  
1              PASADENA         Pasadena    CA   CU  
2  OLIVE VIEW EMPLOYEES           SYLMAR    CA   CU  
3     FARMERS INSURANCE          Burbank    CA   CU  
4                RANCHO           DOWNEY    CA   CU  


In [383]:
# Merge and compare the change over last 2 quarters

merged_df_dep = pd.merge(merged_df_june, merged_df_sep, on="CUNumber", how="inner")
merged_df_dep

,Charter_x,Total Assets_x,Total Shares and Deposits_x,CUNumber,CUName_x,State_x,City_x,Type_x,Charter_y,Total Assets_y,Total Shares and Deposits_y,CUName_y,City_y,State_y,Type_y
0,566,3442694640,2714094872,566,NUVISION,CA,HUNTINGTON BEAC,CU,566,3469113230,2731790560,NUVISION,HUNTINGTON BEAC,CA,CU
1,594,374754091,334915398,594,PASADENA,CA,Pasadena,CU,594,364712289,324358927,PASADENA,Pasadena,CA,CU
2,1034,63640761,49291011,1034,OLIVE VIEW EMPLOYEES,CA,SYLMAR,CU,1034,63443173,48931060,OLIVE VIEW EMPLOYEES,SYLMAR,CA,CU
3,1074,1503029445,1256306639,1074,FARMERS INSURANCE,CA,Burbank,CU,1074,1461148571,1229322187,FARMERS INSURANCE,Burbank,CA,CU
4,1204,107666139,99193332,1204,RANCHO,CA,DOWNEY,CU,1204,108016124,99260328,RANCHO,DOWNEY,CA,CU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,68549,37038781,31548662,68549,MEDIA CITY,CA,BURBANK,CU,68549,38649552,32401070,MEDIA CITY,BURBANK,CA,CU
239,68579,9586191891,8063704676,68579,PATELCO,CA,Dublin,CU,68579,9525458343,7848994798,PATELCO,Dublin,CA,CU
240,68668,834000401,684989575,68668,1ST NORTHERN CALIFORNIA,CA,Martinez,CU,68668,823837786,674249910,1ST NORTHERN CALIFORNIA,Martinez,CA,CU
241,68712,3885613740,3340557844,68712,VALLEY STRONG,CA,BAKERSFIELD,CU,68712,3925605060,3415504790,VALLEY STRONG,BAKERSFIELD,CA,CU


### 1. How many banks and credit unions are active by asset tier (between $500 M and $1B)

In [337]:
df_institutions.head()

,score,data_ZIP,data_CITY,data_ACTIVE,data_STNAME,data_ASSET,data_STALP,data_DEP,data_COUNTY,data_NAME,data_UNINUM,data_ID
0,0,62231,Carlyle,1,Illinois,301003.0,IL,254473.0,Clinton,1NB Bank,9231,14761
1,0,63376,Saint Peters,1,Missouri,181448.0,MO,162837.0,St. Charles,1st Advantage Bank,429739,57899
2,0,74035,Hominy,1,Oklahoma,47473.0,OK,44326.0,Osage,1st Bank in Hominy,2688,4122
3,0,08243,Sea Isle City,1,New Jersey,301759.0,NJ,219399.0,Cape May,1st Bank of Sea Isle City,43201,30367
4,0,85364,Yuma,1,Arizona,577947.0,AZ,518166.0,Yuma,1st Bank Yuma,360755,57298


In [344]:
# Bank Count

filtered_institutions = df_institutions[(df_institutions['data_ASSET'] >= 500000000) & (df_institutions['data_ASSET'] <= 1000000000)]
num_banks = filtered_institutions['data_UNINUM'].nunique()

# Credit Union Count

filtered_cu = merged_df[(merged_df['Total Assets'] >= 500000000) & (merged_df['Total Assets'] <= 1000000000)]
num_cu = filtered_cu['CUNumber'].nunique()

print(f"Number of Active Banks That Have Total Assets Between $500M and $1B Is:  {num_banks}")
print(f"Number of Active Credit Unions That Have Total Assets Between $500M and $1B Is:  {num_cu}")

Number of Active Banks That Have Total Assets Between $500M and $1B Is:  4
Number of Active Credit Unions That Have Total Assets Between $500M and $1B Is:  24


### 2. Which banks and credit unions experienced >5% decline in deposits last quarter?

In [391]:
merged_df_dep = pd.merge(merged_df_june, merged_df_sep, on="CUNumber", how="inner")

merged_df_dep = merged_df_dep.rename(columns={"Total Shares and Deposits_x": "TotDep_June", "Total Shares and Deposits_y": "TotDep_Sep",})

merged_df_dep["Diff_TotDep"] = merged_df_dep["TotDep_Sep"] - merged_df_dep["TotDep_June"]

merged_df_dep["%_Diff"] = 100 * round((merged_df_dep["Diff_TotDep"] / merged_df_dep["TotDep_June"]), 2)

In [392]:
merged_df_dep["%_Diff"] < -5.0

,Charter_x,Total Assets_x,TotDep_June,CUNumber,CUName_x,State_x,City_x,Type_x,Charter_y,Total Assets_y,TotDep_Sep,CUName_y,City_y,State_y,Type_y,Diff_TotDep,%_Diff
0,566,3442694640,2714094872,566,NUVISION,CA,HUNTINGTON BEAC,CU,566,3469113230,2731790560,NUVISION,HUNTINGTON BEAC,CA,CU,17695688,1.0
1,594,374754091,334915398,594,PASADENA,CA,Pasadena,CU,594,364712289,324358927,PASADENA,Pasadena,CA,CU,-10556471,-3.0
2,1034,63640761,49291011,1034,OLIVE VIEW EMPLOYEES,CA,SYLMAR,CU,1034,63443173,48931060,OLIVE VIEW EMPLOYEES,SYLMAR,CA,CU,-359951,-1.0
3,1074,1503029445,1256306639,1074,FARMERS INSURANCE,CA,Burbank,CU,1074,1461148571,1229322187,FARMERS INSURANCE,Burbank,CA,CU,-26984452,-2.0
4,1204,107666139,99193332,1204,RANCHO,CA,DOWNEY,CU,1204,108016124,99260328,RANCHO,DOWNEY,CA,CU,66996,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,68549,37038781,31548662,68549,MEDIA CITY,CA,BURBANK,CU,68549,38649552,32401070,MEDIA CITY,BURBANK,CA,CU,852408,3.0
239,68579,9586191891,8063704676,68579,PATELCO,CA,Dublin,CU,68579,9525458343,7848994798,PATELCO,Dublin,CA,CU,-214709878,-3.0
240,68668,834000401,684989575,68668,1ST NORTHERN CALIFORNIA,CA,Martinez,CU,68668,823837786,674249910,1ST NORTHERN CALIFORNIA,Martinez,CA,CU,-10739665,-2.0
241,68712,3885613740,3340557844,68712,VALLEY STRONG,CA,BAKERSFIELD,CU,68712,3925605060,3415504790,VALLEY STRONG,BAKERSFIELD,CA,CU,74946946,2.0


In [395]:
print(f"Banks and Credit Unions that have experienced > 5% decline in deposits last quarter are")

merged_df_dep[(merged_df_dep["%_Diff"] < -5.0)][["CUName_x", "City_x", "State_x"]]

Banks and Credit Unions that have experienced > 5% decline in deposits last quarter are


,CUName_x,City_x,State_x
33,SKYONE,Hawthorne,CA
43,MATTEL,El Segundo,CA
72,SANTA MARIA ASSOCIATED EMPLOYEES,Santa Maria,CA
80,ANTIOCH COMMUNITY,Antioch,CA
106,BOURNS EMPLOYEES,Riverside,CA
116,DELANCEY STREET,SAN FRANCISCO,CA
151,BLUPEAK,San Diego,CA
171,VISION ONE,Sacramento,CA
188,JONES METHODIST CHURCH,SAN FRANCISCO,CA
235,FRONTWAVE,Oceanside,CA
